In [1]:
import os

# = = =  = = =  = = =

import pandas as pd

# = = =  = = =  = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = =  = = =  = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = =  = = =  = = =

import json

# = = =  = = =  = = =

import numpy as np

# = = =  = = =  = = =

import re
import html as html_lib

In [2]:
directory_in = './file/1.part_number'
list_file = [file for file in sorted(list(os.walk(directory_in))[0][2]) if file not in ['.DS_Store'] and os.path.isfile(os.path.join(directory_in, file))]
file = list_file[0]

file

'1.xlsx'

In [3]:
print('Data Loading...')
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(os.path.join(directory_in, file),
                                                                       header=0,
                                                                       dtype=str).fillna('')
print('Loading Done !')
print()

df_input['No'] = df_input['No'].astype(int)

crawler_remain = len(df_input)

print(f'总数量：{crawler_remain}')
print()

df_input

Data Loading...
Loading Done !

总数量：353



,No,Group,Type,Manufacturer,Part Number,Url,Type Code
0,1,Interior,Accelerator Pedal Position Sensor,STANDARD MOTOR PRODUCTS,APK100,https://www.rockauto.com/en/parts/standard+mot...,5061
1,2,Interior,Accelerator Pedal Position Sensor,STANDARD MOTOR PRODUCTS,APK101,https://www.rockauto.com/en/parts/standard+mot...,5061
2,3,Interior,Accelerator Pedal Position Sensor,STANDARD MOTOR PRODUCTS,APK102,https://www.rockauto.com/en/parts/standard+mot...,5061
3,4,Interior,Accelerator Pedal Position Sensor,STANDARD MOTOR PRODUCTS,APK103,https://www.rockauto.com/en/parts/standard+mot...,5061
4,5,Interior,Accelerator Pedal Position Sensor,STANDARD MOTOR PRODUCTS,APS101,https://www.rockauto.com/en/parts/standard+mot...,5061
...,...,...,...,...,...,...,...
348,349,Interior,Accelerator Pedal Position Sensor,STANDARD MOTOR PRODUCTS,APS713,https://www.rockauto.com/en/parts/standard+mot...,5061
349,350,Interior,Accelerator Pedal Position Sensor,STANDARD MOTOR PRODUCTS,APS714,https://www.rockauto.com/en/parts/standard+mot...,5061
350,351,Interior,Accelerator Pedal Position Sensor,STANDARD MOTOR PRODUCTS,APS716,https://www.rockauto.com/en/parts/standard+mot...,5061
351,352,Interior,Accelerator Pedal Position Sensor,STANDARD MOTOR PRODUCTS,APS717,https://www.rockauto.com/en/parts/standard+mot...,5061


In [4]:
crawler_status, (crawler_index, crawler_series) = False, next(df_input.iterrows())

crawler_status, crawler_index, crawler_series

(False,
 0,
 No                                                              1
 Group                                                    Interior
 Type                            Accelerator Pedal Position Sensor
 Manufacturer                              STANDARD MOTOR PRODUCTS
 Part Number                                                APK100
 Url             https://www.rockauto.com/en/parts/standard+mot...
 Type Code                                                    5061
 Name: 0, dtype: object)

In [5]:
import time

data = {'dopartsearch': '1',
        'partsearch[partnum][partsearch_007]': crawler_series['Part Number'],
        'partsearch[manufacturer][partsearch_007]': crawler_series['Manufacturer'],
        'partsearch[parttype][partsearch_007]': crawler_series['Type Code'],
        'partsearch[do][partsearch_007]': '1',
        'func': 'sendparttabsearch',
        'payload': '{}',
        'api_json_request': '1'}

crawler_retry = 0
while True:
    crawler_retry += 1
    print(crawler_retry)
    try:
        url_check = crawler_series['Url']

        resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                             data=data,
                             headers=get_header(),
                             proxies=get_proxy(),
                             timeout=(5, 15))

        if resp.status_code == 200:
            dict_ = resp.json()

            if 'redirect_to_url' not in dict_ and 'collected_javascript' in dict_:
                break
    except KeyboardInterrupt:
        break
    except:
        time.sleep(3)
        continue

dict_

1


{'searchnoderesults': '<div class="ranavnode" id="nav[__GIP__1__]"><input type="hidden" autocomplete="off" id="jsn[__GIP__1__]" value="{&quot;groupindex&quot;:&quot;__GIP__1__&quot;,&quot;tab&quot;:&quot;partsearch&quot;,&quot;idepth&quot;:0,&quot;parttype&quot;:&quot;5061&quot;,&quot;catalogname&quot;:&quot;STANDARD MOTOR PRODUCTS&quot;,&quot;partnumber&quot;:&quot;APK100&quot;,&quot;nodetype&quot;:&quot;parttabsearch&quot;,&quot;expanded&quot;:true,&quot;loaded&quot;:true}" /><input type="hidden" autocomplete="off" name="navnodeunique[partsearch-5061]" id="navnodeunique[partsearch-5061]" value="__GIP__1__" /><div id="nav_o[__GIP__1__]" class="ranavouter"><div class="inner"><table class="tbl"  cellpadding="0" cellspacing="0"><tr><td class="niconspace nexpandedicon" id="navicon[__GIP__1__]"><a href="/en/parts/standard+motor+products,APK100,accelerator+pedal+position+sensor,5061" onclick="cataloglite.LinkIntercept_ToggleNavNode(&quot;__GIP__1__&quot;); return false;" tabindex="-1"><img 

In [6]:
soup = BeautifulSoup(dict_['searchnoderesults'], 'lxml')
html = etree.HTML(str(soup))

with open('./part.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<html>
 <body>
  <div class="ranavnode" id="nav[__GIP__1__]">
   <input autocomplete="off" id="jsn[__GIP__1__]" type="hidden" value='{"groupindex":"__GIP__1__","tab":"partsearch","idepth":0,"parttype":"5061","catalogname":"STANDARD MOTOR PRODUCTS","partnumber":"APK100","nodetype":"parttabsearch","expanded":true,"loaded":true}'/>
   <input autocomplete="off" id="navnodeunique[partsearch-5061]" name="navnodeunique[partsearch-5061]" type="hidden" value="__GIP__1__"/>
   <div class="ranavouter" id="nav_o[__GIP__1__]">
    <div class="inner">
     <table cellpadding="0" cellspacing="0" class="tbl">
      <tr>
       <td class="niconspace nexpandedicon" id="navicon[__GIP__1__]">
        <a href="/en/parts/standard+motor+products,APK100,accelerator+pedal+position+sensor,5061" onclick='cataloglite.LinkIntercept_ToggleNavNode("__GIP__1__"); return false;' tabindex="-1">
         <img alt="Toggle" src="/Images/1pxtransparent.png"/>
        </a>
       </td>
       <td class="nlabel">
        <a 

In [7]:
list_tbody = html.xpath('//tbody[contains(@id, "listingcontainer[__GIP__")]')

list_tbody

[<Element tbody at 0x12192b300>]

In [8]:
list_dict = [json.loads(tbody.xpath('./tr[1]/td[contains(@class, "listing-inner-content")]/input[contains(@id, "listing_data_essential[__GIP__")]/@value')[0]) for tbody in list_tbody]

list_dict

[{'groupindex': '__GIP__3__',
  'carcode': 0,
  'parttype': '5061',
  'partkey': '12769033',
  'opts': {'0-0-0-1': {'warehouse': '92407',
    'whpartnum': 'SI APK100',
    'optionlist': '0',
    'paramcode': '0',
    'notekey': '0',
    'multiple': '1'}}}]

In [9]:
list_type_code, list_part_code = [dict_['parttype'] for dict_ in list_dict], [dict_['partkey'] for dict_ in list_dict]

list_type_code, list_part_code

(['5061'], ['12769033'])

In [10]:
array_index_type_code = np.where(np.array(list_type_code) == crawler_series['Type Code'])[0]

array_index_type_code

array([0])

In [11]:
list_dict_2 = [json.loads(tbody.xpath('./tr[1]/td/input[contains(@id, "listing_data_supplemental[__GIP__")]/@value')[0]) for tbody in list_tbody]

list_dict_2

[{'partnumber': 'APK100',
  'catalogname': 'STANDARD MOTOR PRODUCTS',
  'belongstolisting': '__GIP__2__',
  'sortgroup': 0,
  'sortgrouptext': '',
  'paramdesc': ''}]

In [12]:
list_part_number, list_manufacturer, list_choose = [dict_['partnumber'] for dict_ in list_dict_2], [dict_['catalogname'] for dict_ in list_dict_2], [dict_['paramdesc'].strip() for dict_ in list_dict_2]

list_part_number, list_manufacturer, list_choose

(['APK100'], ['STANDARD MOTOR PRODUCTS'], [''])

In [13]:
array_index_part_number, array_index_manufacturer = np.where(np.array(list_part_number) == crawler_series['Part Number'])[0], np.where(np.array(list_manufacturer) == crawler_series['Manufacturer'])[0]

array_index_part_number, array_index_manufacturer

(array([0]), array([0]))

In [14]:
array_index_part = np.intersect1d(np.intersect1d(array_index_type_code, array_index_part_number), array_index_manufacturer)

array_index_part

array([0])

In [15]:
if array_index_part.size == 1:
    index_part = array_index_part[0]

index_part

0

In [16]:
list_span = list_tbody[index_part].xpath('./tr[1]/td[contains(@class, "listing-inner-content")]/div[@class="listing-text-row-moreinfo-truck"]/span[@class="span-link-underline-remover"]')
list_note = [etree.tostring(span).decode('utf-8') for span in list_span]
list_note = [re.sub(r'<.*?>', '', note) for note in list_note]
list_note = [re.sub(r' +', ' ', note.strip()) for note in list_note]
list_note = [html_lib.unescape(note) for note in list_note]

dict_note, i = {}, 0
for note in list_note:
    if note.strip() != '':
        dict_note[str(i)] = note
        i += 1

dict_note

{}

In [17]:
list_a = list_tbody[index_part].xpath('./tr[1]/td[contains(@class, "listing-inner-content")]/div[@class="listing-text-row-moreinfo-truck"]/a[@class="ra-btn ra-btn-moreinfo"]')
info = list_a[0].xpath('./@href')[0].strip() if list_a else ''
info = info if info.startswith('https://www.rockauto.com/en/moreinfo.php?') else f'https://www.rockauto.com/en/moreinfo.php?pk={list_part_code[index_part]}&pt={list_type_code[index_part]}&Lennon=1'

info

'https://www.rockauto.com/en/moreinfo.php?pk=12769033&cc=0&pt=5061'

In [18]:
list_div = list_tbody[index_part].xpath('./tr[1]/td[contains(@class, "listing-inner-content")]/div[@class="listing-text-row"]/span[contains(@title, "Please make sure")]/parent::div[@class="listing-text-row"]')
list_note = [etree.tostring(div).decode('utf-8') for div in list_div]
list_note = [re.sub(r'<.*?>', '', note) for note in list_note]
list_note = [re.sub(r' +', ' ', note.strip()) for note in list_note]
list_note = [html_lib.unescape(note) for note in list_note]

dict_note_2, i = {}, 0
for note in list_note:
    if note.strip() != '':
        dict_note_2[str(i)] = note
        i += 1

dict_note_2

{'0': 'Category: Accelerator Pedal Position Sensor'}

In [19]:
list_list_text = [text.split(',') for td in list_tbody[index_part].xpath('./tr[1]/td[contains(@class, "listing-inner-content")]') for text in td.xpath('./span[contains(@title, "Replaces these Alternate")]/text()')]
list_text = [text.strip() for list_text in list_list_text for text in list_text if text.strip() != '']
oem = ';'.join(list(dict.fromkeys(list_text)))

oem

'15177923RK'

In [20]:
list_input = list_tbody[index_part].xpath('./tr[1]/td/div[contains(@class, "listing-image-space-occupy")]/input[contains(@id, "jsninlineimg[__GIP__")]')
list_dict = [json.loads(value) for input_ in list_input for value in input_.xpath('./@value')]

dict_src, i = {}, 0
for dict_ in list_dict:
    list_src = dict_.get('Slots', [])
    for src in list_src:
        dict_src[str(i)] = f'''https://www.rockauto.com{src['ImageData']['Full'].strip()}'''
        i += 1

dict_src

{'0': 'https://www.rockauto.com/info/154/APK100_Front.jpg',
 '1': 'https://www.rockauto.com/info/154/APK100_Back.jpg',
 '2': 'https://www.rockauto.com/info/154/APK100_Other.jpg'}

In [21]:
list_span = list_tbody[index_part].xpath('./tr[1]/td[contains(@id, "listingtd[__GIP__") and contains(@id, "__][price]")]/span[contains(@id, "dprice[__GIP__") and contains(@id, "__][td]")]/span/span[contains(@id, "dprice[__GIP__") and contains(@id, "__][v]")]')
list_price = [etree.tostring(span).decode('utf-8') for span in list_span]
list_price = [re.sub(r'<.*?>', '', price) for price in list_price]
list_price = [re.sub(r' +', ' ', price.strip()) for price in list_price]
list_price = [html_lib.unescape(price) for price in list_price]
dict_price = {'0': '; '.join(list_price)}

dict_price

{'0': '$32.79'}

In [22]:
list_td = list_tbody[index_part].xpath('./tr[1]/td[contains(@class, "listing-inner-content")]')
list_price = [etree.tostring(option).decode('utf-8') for td in list_td for option in td.xpath('./div/div[contains(@class, "listing-optionchoice-multiple-container")]/span[contains(@id, "ddrepl[optionchoice[__GIP__") and contains(@id, "__]][container]")]/span[@class="hide-if-js"]/select[contains(@id, "optionchoice[__GIP__") and contains(@id, "__]")]/option[@value!=""]')]
list_price = [re.sub(r'<.*?>', '', price) for price in list_price]
list_price = [re.sub(r' +', ' ', price.strip()) for price in list_price]
list_price = [html_lib.unescape(price) for price in list_price]
if list_price:
    dict_price['1'] = {}
    for i, price in enumerate(list_price):
        dict_price['1'][str(i)] = price

dict_price

{'0': '$32.79'}

In [23]:
list_dict_correct.append({'No': crawler_series['No'],
                          'Group': crawler_series['Group'],
                          'Type': crawler_series['Type'],
                          'Manufacturer': crawler_series['Manufacturer'],
                          'Part Number': crawler_series['Part Number'],
                          'OEM': oem,
                          'Picture': '',
                          'Url': crawler_series['Url'],
                          'Json_Src': json.dumps(dict_src),
                          'Info': info,
                          'Json_Note 1': json.dumps(dict_note),
                          'Json_Note 2': json.dumps(dict_note_2),
                          'Choose': list_choose[index_part],
                          'Json_Price': json.dumps(dict_price),
                          'Type Code': crawler_series['Type Code'],
                          'Part Code': list_part_code[index_part],
                          'JOIN_MPNTCPC': f'''{crawler_series['Manufacturer']};{crawler_series['Part Number']};{crawler_series['Type Code']};{list_part_code[index_part]}'''})

list_dict_correct

[{'No': 1,
  'Group': 'Interior',
  'Type': 'Accelerator Pedal Position Sensor',
  'Manufacturer': 'STANDARD MOTOR PRODUCTS',
  'Part Number': 'APK100',
  'OEM': '15177923RK',
  'Picture': '',
  'Url': 'https://www.rockauto.com/en/parts/standard+motor+products,APK100,accelerator+pedal+position+sensor,5061',
  'Json_Src': '{"0": "https://www.rockauto.com/info/154/APK100_Front.jpg", "1": "https://www.rockauto.com/info/154/APK100_Back.jpg", "2": "https://www.rockauto.com/info/154/APK100_Other.jpg"}',
  'Info': 'https://www.rockauto.com/en/moreinfo.php?pk=12769033&cc=0&pt=5061',
  'Json_Note 1': '{}',
  'Json_Note 2': '{"0": "Category: Accelerator Pedal Position Sensor"}',
  'Choose': '',
  'Json_Price': '{"0": "$32.79"}',
  'Type Code': '5061',
  'Part Code': '12769033',
  'JOIN_MPNTCPC': 'STANDARD MOTOR PRODUCTS;APK100;5061;12769033'}]

In [24]:
crawler_status = True

crawler_status

True

In [25]:
crawler_series['Check_Url'] = url_check

list_series_error.append(crawler_series)

list_series_error

[No                                                              1
 Group                                                    Interior
 Type                            Accelerator Pedal Position Sensor
 Manufacturer                              STANDARD MOTOR PRODUCTS
 Part Number                                                APK100
 Url             https://www.rockauto.com/en/parts/standard+mot...
 Type Code                                                    5061
 Request_Url     https://www.rockauto.com/en/parts/standard+mot...
 Name: 0, dtype: object]

In [26]:
print('Data Outputting...')
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['No'],
                                                             ascending=[True],
                                                             ignore_index=True)
    df_correct.to_excel('./test_part.xlsx', index=False)

df_correct if list_dict_correct else ''

Data Outputting...


,No,Group,Type,Manufacturer,Part Number,OEM,Picture,Url,Json_Src,Info,Json_Note 1,Json_Note 2,Choose,Json_Price,Type Code,Part Code,JOIN_MPNTCPC
0,1,Interior,Accelerator Pedal Position Sensor,STANDARD MOTOR PRODUCTS,APK100,15177923RK,,https://www.rockauto.com/en/parts/standard+mot...,"{""0"": ""https://www.rockauto.com/info/154/APK10...",https://www.rockauto.com/en/moreinfo.php?pk=12...,{},"{""0"": ""Category: Accelerator Pedal Position Se...",,"{""0"": ""$32.79""}",5061,12769033,STANDARD MOTOR PRODUCTS;APK100;5061;12769033


In [27]:
print('Output Finished !')
print()

Output Finished !



In [28]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./test_part_error.xlsx', index=False)
    print('- Error Occurred -')
    print()

df_error if list_series_error else ''

- Error Occurred -



,No,Group,Type,Manufacturer,Part Number,Url,Type Code,Request_Url
0,1,Interior,Accelerator Pedal Position Sensor,STANDARD MOTOR PRODUCTS,APK100,https://www.rockauto.com/en/parts/standard+mot...,5061,https://www.rockauto.com/en/parts/standard+mot...


In [29]:
print('Done ~')

Done ~
